In [ ]:
!pip install pyshp
!pip install requests # For API call
!pip install geopandas
!pip install geoplot
!pip install mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 2.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 7.4 MB/s et

In [ ]:
import shapefile as shp
import requests as rq
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import sys
import itertools
import warnings
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc
from shapely.geometry import Point, Polygon
import pyproj

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
#load data
sf_city = shp.Reader("/gdrive/My Drive/CS506-DS/data/City_Council_Districts_-_Effective_for_the_2023_Municipal_Election.shp")
sf_community = shp.Reader("/gdrive/My Drive/CS506-DS/data/Community_Centers.shp")

In [ ]:
fields = [x[0] for x in sf_city.fields][1:]
records = [list(i) for i in sf_city.records()]
shps = [s.points for s in sf_city.shapes()]
city_councils = pd.DataFrame(columns=fields, data=records)
city_councils = city_councils.assign(coords=shps)
print(city_councils)

fields = [x[0] for x in sf_community.fields][1:]
records = [list(i) for i in sf_community.records()]
shps = [s.points for s in sf_community.shapes()]
community_centers = pd.DataFrame(columns=fields, data=records)
community_centers = community_centers.assign(coords=shps)
print(community_centers)

   FID    LONGNAME SHORTNAME  DISTRICT       COLOR  TOTAL  TARGET_DEV  \
0    1  District 1        D1         1  -851848832  76830        1758   
1    2  District 2        D2         2   465990528  77466        2394   
2    3  District 3        D3         3   494969216  73285       -1787   
3    4  District 4        D4         4      -65408  72917       -2155   
4    5  District 5        D5         5   412292224  75436         364   
5    6  District 6        D6         6  1971620992  76523        1451   
6    7  District 7        D7         7  1946148736  72147       -2925   
7    8  District 8        D8         8   639511936  76370        1298   
8    9  District 9        D9         9  -119742707  74673        -399   

   TARGET_D_1  TOTAL18  TOTALHISP  ...  BLACK  AIAN  ASIAN  HPI  OTHER  MLTMN  \
0        2.34    64257      24552  ...   3637   783   5080   50  20935   1689   
1        3.19    70183       6640  ...   4520   269  13604   55   4893   1005   
2       -2.38    60834    

In [ ]:
# Define projection systems
wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 coordinate system
web_mercator = pyproj.CRS('EPSG:3857')  # Web Mercator projection
city_council_id=[]
# Loop through community centers dataset
for index, row in community_centers.iterrows():
    # Convert point coordinates to WGS84
    transformer = pyproj.Transformer.from_crs(web_mercator, wgs84, always_xy=True)
    lon, lat = transformer.transform(row['coords'][0][0], row['coords'][0][1])
    point_wgs84 = Point(lon, lat)
    
    # Check if point is inside city council polygons
    council_index = None
    for i, coords in enumerate(city_councils['coords']):
        polygon = Polygon(coords)
        if polygon.contains(point_wgs84):
            council_index = i
            break
    if council_index is not None:
        council_id = city_councils.loc[council_index, 'DISTRICT']
        city_council_id.append(council_id)
        print(f"Community center {row['SITE']} is inside city council {council_id}.")
    else:
        city_council_id.append(-1)
        print(f"Community center {row['SITE']} is not inside any city council shape.")


Community center Pino is inside city council 1.
Community center Paris Street Pool* is inside city council 1.
Community center Paris Street is inside city council 1.
Community center Golden Age Center is inside city council 1.
Community center Charlestown* is inside city council 1.
Community center Nazzaro is inside city council 1.
Community center Quincy* is inside city council 2.
Community center Condon* is inside city council 2.
Community center Tynan is inside city council 2.
Community center Curley is inside city council 2.
Community center Vine Street is inside city council 7.
Community center Blackstone* is inside city council 2.
Community center Rec. Division @ Madison Park* is inside city council 7.
Community center Tobin is inside city council 8.
Community center Hennigan* is inside city council 6.
Community center Shelburne is inside city council 7.
Community center Holland* is inside city council 4.
Community center Perkins* is inside city council 4.
Community center Clevel

In [ ]:
sf_community = shp.Reader("/gdrive/My Drive/CS506-DS/data/Community_Centers.shp")
fields = [x[0] for x in sf_community.fields][1:]
records = [list(i) for i in sf_community.records()]
shps = [s.points for s in sf_community.shapes()]
geometry = [Point(xy) for xy in shps]
community_centers = community_centers.assign(geometry=geometry)
community_centers = community_centers.assign(city_council_id=city_council_id)

print(community_centers)

# Create new GeoDataFrame with Point geometry
gdf = gpd.GeoDataFrame(community_centers, geometry=geometry, crs='EPSG:3857')
# Save GeoDataFrame to shapefile
gdf.to_file('community_centers_city_council.shp', driver='ESRI Shapefile')
community_centers.to_csv('community_centers_city_council.csv', index=False)

    FID  OBJECTID                           SITE     PHONE       FAX  \
0     1       0.0                           Pino  635-5120  635-1874   
1     2       0.0             Paris Street Pool*  635-1410  635-5079   
2     3       0.0                   Paris Street  635-5125  635-5079   
3     4       0.0              Golden Age Center  635-5175  635-5647   
4     5       0.0                   Charlestown*  635-5169  635-5084   
5     6       0.0                        Nazzaro  635-5166  635-5053   
6     7       0.0                        Quincy*  635-5129  635-5132   
7     8       0.0                        Condon*  635-5100  635-5103   
8     9       0.0                          Tynan  635-5110  635-5065   
9    10       0.0                         Curley  635-5104  635-1259   
10   11       0.0                    Vine Street  635-1285  635-1291   
11   12       0.0                    Blackstone*  635-5162  635-5057   
12   13       0.0  Rec. Division @ Madison Park*  635-5206  635-